In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import zipfile
import os

In [60]:
path ='C:/Users/user'
os.chdir(path)
chromedriver = 'chromedriver.exe' 
driver = webdriver.Chrome()
driver.implicitly_wait(3) 

In [ ]:
# selenium 다운로드 저장경로 수정 해야됨

In [61]:
# 로그인하기
driver.get('https://data.kma.go.kr/cmmn/main.do')
time.sleep(2)

login="""//*[@id="loginBtn"]"""
driver.find_element_by_xpath(login).click()
time.sleep(2)


tag_id = driver.find_element_by_name('loginId')
tag_pw = driver.find_element_by_name('passwordNo')
tag_id.clear()
tag_pw.clear()
time.sleep(1)

# id입력
tag_id.click()
tag_id.send_keys('hyoik123@gmail.com')
time.sleep(1)

# 비밀번호 입력
tag_pw.click()
tag_pw.send_keys('hyoik123!')
time.sleep(1)

login1="""//*[@id="loginbtn"]"""
driver.find_element_by_xpath(login1).click()
time.sleep(2)

In [62]:
# 데이터 들어가기
wts="""//*[@id="mwrap"]/div[3]/div/ul/li[2]/a"""
driver.find_element_by_xpath(wts).click()
time.sleep(2)

# 초단기실황 들어가기
wts1="""//*[@id="leftMenuUi"]/li[4]/a/span"""
driver.find_element_by_xpath(wts1).click()
time.sleep(2)

In [63]:
html = driver.page_source
bs=BeautifulSoup(html,'html.parser')

In [64]:
# 선택초기화

seoul="""//*[@id="ztree_2_switch"]"""
driver.find_element_by_xpath(seoul).click()
time.sleep(2)

seoul1="""//*[@id="ztree_2_check"]"""
driver.find_element_by_xpath(seoul1).click()
time.sleep(2)

elem="""//*[@id="ztree1_2_check"]"""
driver.find_element_by_xpath(elem).click()
time.sleep(2)

In [65]:
#임시 요소 저장 ex(습도, 기온, 풍속, 강수) 각 요소 xpath 긁어와야함
elem1="""//*[@id="ztree1_4_check"]"""#해당 xpath-> 습도
elem2="습도"
driver.find_element_by_xpath(elem1).click()
#time.sleep(2)


#연도 구하기 ex) 1 = 2020년, 2 = 2019년
year_list=['3']

for year in year_list:
    
    st_year='//*[@id="startDt"]/option['+year+']'
    driver.find_element_by_xpath(st_year).click()
    #time.sleep(1)
    
    end_year='//*[@id="endDt"]/option['+year+']'
    driver.find_element_by_xpath(end_year).click()
    #time.sleep(1)
    #달 구하기
    month_list=['1','2','3','4','5','6','7','8','9','10','11','12']
    for month in month_list:
        
        st_month='//*[@id="startMt"]/option['+month+']'
        driver.find_element_by_xpath(st_month).click()
#         time.sleep(1)
        
        end_month='//*[@id="endMt"]/option['+month+']'
        driver.find_element_by_xpath(end_month).click()
#         time.sleep(1)
        #구 구하기 ex) 3 = 종로구, 21 = 중구
        gu_list=['3']
        

        # 자치구 선택
        for gu in gu_list:
            
            gu_xp='//*[@id="ztree_'+gu+'_switch"]'
            driver.find_element_by_xpath(gu_xp).click()
#             time.sleep(1) 
    
            # 첫번째 동선택 (해당 자치구의 첫번째 동 선택)
            gu1=int(gu)
            gu2=str(gu1+1)
            dong_xp='//*[@id="ztree_'+gu2+'_check"]'
            driver.find_element_by_xpath(dong_xp).click()
#             time.sleep(1)
    
    
            #조회
            search="""//*[@id="dsForm"]/div[3]/a"""
            driver.find_element_by_xpath(search).click()
#             time.sleep(1)
    
            #전체선택
            check="""//*[@id="checkAll"]"""
            driver.find_element_by_xpath(check).click()
#             time.sleep(1)
    
    
            #다운로드
            down="""//*[@id="content"]/div[3]/div[4]/div/a"""
            driver.find_element_by_xpath(down).click()
#             time.sleep(1)
    
    
            #환경선택
            choice=driver.find_element_by_css_selector('#requestForm > ul > li:nth-child(4) > input[type=radio]:nth-child(7)')
            choice.click()
            #요청
            time.sleep(2)
            req=driver.find_element_by_css_selector('#btnArea > input.btn.btn-primary')
            req.click()
            time.sleep(2)

            path = "C:/Users/user/Downloads"
            file_list = os.listdir(path)
            print(file_list)
            

            os.chdir(path)
            weatherzip=zipfile.ZipFile(file_list[0])
            weatherzip.extractall()
            weatherzip.close()
            file_list1=os.listdir(path)
            
            jachigu='#ztree_'+gu+'_span'
            jachigu1=bs.select(jachigu)[0].text
            print(bs.select(jachigu))
            print(jachigu1)
            year1=2021-int(year)
            year2=str(year1)+'0000'
            month1=int(month)*100
            date=int(year2)+month1
            wt=pd.read_csv(file_list1[0], encoding='utf-8')
            wt=wt.rename(columns={wt.columns[0]:'day',wt.columns[2]:elem2})
            del wt['hour']
            wt1=wt.groupby(['day'], as_index=False).mean()
            wt1['날짜']=date + (wt1['day'])
            wt1['자치구']=jachigu1
            del wt1["day"]
            date1=str(date)+'.csv'
            date2='C:/Users/user\Documents/wheather/'+date1
            wt1.to_csv(date2,header=True, encoding='utf-8-sig')
            
            path = "C:/Users/user/Downloads"
            file_list = os.listdir(path)
            file1='C:/Users/user/Downloads/' + file_list1[0]
            file2="C:/Users/user/Downloads/"+ file_list1[1]
            os.remove(file1)
            os.remove(file2)
            # 해야될 일
            #지정 폴더 알집파일 알집풀기 
            #해당 csv 읽기
            #전처리 (하루 평균 구하기, 날짜 열 부여, 불필요 열 삭제)
            #파일 이름지정하여 저장(연도와 월 리스트 값 이용예정)
            #지정 파일 속 알집파일 삭제
            
            
        #자치구 초기화
        gu_end='//*[@id="ztree_'+gu+'_check"]'
        driver.find_element_by_xpath(gu_end).click()
        time.sleep(2)
    
        gu_xp='//*[@id="ztree_'+gu+'_switch"]'
        driver.find_element_by_xpath(gu_xp).click()
        time.sleep(2) 

['20200424230225.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230237.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230248.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230259.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230310.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230321.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230331.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230342.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230353.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230404.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230415.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구
['20200424230427.zip', 'desktop.ini']
[<span id="ztree_3_span">종로구</span>]
종로구


In [7]:
# gu_list=['3']

# # 자치구 선택
# for gu in gu_list: 
#     gu_xp='//*[@id="ztree_'+gu+'_switch"]'
#     driver.find_element_by_xpath(gu_xp).click()
#     time.sleep(1) 
    
#     # 첫번째 동선택
#     gu1=int(gu)
#     gu2=str(gu1+1)
#     dong_xp='//*[@id="ztree_'+gu2+'_check"]'
#     driver.find_element_by_xpath(dong_xp).click()
#     time.sleep(1)
    
#     #임시 요소 저장
#     elem1="""//*[@id="ztree1_4_check"]"""
#     driver.find_element_by_xpath(elem1).click()
#     time.sleep(2)
    
#     #조회
#     search="""//*[@id="dsForm"]/div[3]/a"""
#     driver.find_element_by_xpath(search).click()
#     time.sleep(1)
    
#     #전체선택
#     check="""//*[@id="checkAll"]"""
#     driver.find_element_by_xpath(check).click()
#     time.sleep(1)
    
    
#     #다운로드
#     down="""//*[@id="content"]/div[3]/div[4]/div/a"""
#     driver.find_element_by_xpath(down).click()
#     time.sleep(1)
    
    
#     #환경선택
#     choice=driver.find_element_by_css_selector('#requestForm > ul > li:nth-child(4) > input[type=radio]:nth-child(7)')
#     choice.click()
#     #요청
#     req=driver.find_element_by_css_selector('#btnArea > input.btn.btn-primary')
#     req.click()

    
    
    
    
#     #자치구 초기화
#     gu_end='//*[@id="ztree_'+gu+'_check"]'
#     driver.find_element_by_xpath(gu_end).click()
#     time.sleep(2)
    
#     gu_xp='//*[@id="ztree_'+gu+'_switch"]'
#     driver.find_element_by_xpath(gu_xp).click()
#     time.sleep(2) 